## Extract sector table data from etfdb.com

In [18]:
import pandas as pd

In [19]:
url = 'https://etfdb.com/etfs/sector/'
df = pd.read_html(url)[0].drop([11], axis=0)

In [20]:
sectors = list(df['Sector'])
sectors = [x.lower() for x in sectors]
sectors = [x.replace(' ', '-') for x in sectors]
sectors = [x.replace('discretionary', 'discretionaries') for x in sectors]
sectors

['energy',
 'consumer-discretionaries',
 'financials',
 'healthcare',
 'industrials',
 'technology',
 'materials',
 'consumer-staples',
 'telecom',
 'utilities',
 'real-estate']

## Retrieve Top ETFs from Each Sector

In [ ]:
top_etfs = {}
for s in sectors:
    sector_url = url + s + '/'
    sector_data = pd.read_html(sector_url)[0]
    etf_symbol = sector_data.loc[0, 'Symbol']
    etf_name = sector_data.loc[0, 'ETF Name']
    etf_industry = sector_data.loc[0, 'Industry']
    etf_aum = sector_data.loc[0, 'Total Assets ($MM)']
    top_etfs[s] = {}
    top_etfs[s]['symbol'] = etf_symbol
    top_etfs[s]['name'] = etf_name
    top_etfs[s]['industry'] = etf_industry
    top_etfs[s]['aum'] = etf_aum

top_etfs

## Calculate Sector Momentum

## Final Output in DataFrame

In [ ]:
convert_df = {'Symbol': [], 'Sector': [], 'Industry': []}
for etf in top_etfs.keys():
    convert_df['Symbol'].append(top_etfs[etf]['symbol'])
    convert_df['Sector'].append(etf)
    convert_df['Industry'].append(top_etfs[etf]['industry'])
    # convert_df['Momentum'].append(top_etfs[etf]['symbol'])

df = pd.DataFrame(convert_df)
df